# Random Forest with MARTi data 

https://docs.google.com/document/d/1uldwa4ep4Pr1NrD6hmcmLY3AL-vCiniPYVSswhX1kF8/edit

Last editted - 29th July 2024

In [50]:
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Loading in and polishing data

Using summarised data and performing the analysis at different taxonomic levels

In [19]:
#Loading in the data
raw_data = pd.read_csv('../samp_comp_0624_marti/samp_comp_summed_0624.csv')
# Melt the raw_data dataframe
raw_data_long = raw_data.melt(id_vars=['Taxon', 'NCBI_ID', 'Rank'], var_name='Sample_ID', value_name='Count')

metadata = pd.read_csv('../old_parameters_MARTI_samp_comp_read_data/Phyloseq_data/Sample_table.csv')

# Merge the melted raw_data with seq_info on 'Sample_ID'
merged_data = pd.merge(raw_data_long, metadata, on='Sample_ID', how='left')


In [38]:
#Filter to one taxonomic rank
filtered_data = merged_data[(merged_data['Rank'] == 'genus') &
                            (merged_data['Count'] > 0) #need this for aggregation to work
                            ]

### Aggregating the data

Creating a summary for each Sample, number of unique taxa

In [46]:
# Aggregate to count unique taxa per Sample
data = filtered_data.groupby('Sample_ID').agg({
    'Taxon': pd.Series.nunique, #This is giving the same for each, think I need to remove count = 0 first
    'Location': 'first',
    'Sampler': 'first',
    'Repeat': 'first',
    'Flow_rate': 'first',
    'Sample_length': 'first',
    'Air_volume': 'first',
    'NumReads': 'first',
    'MeanLength': 'first',
    'N50Length': 'first',
    'Shortest': 'first',
    'Longest': 'first',
    'DNA_yield': 'first',
    'Insect': 'first'
}).reset_index().rename(columns={'Taxon': 'num_unique_taxa'})

### Making variables readable by the model

In [49]:
data = data.replace(',','', regex=True)

data.Location = data.Location.astype('category')# Convert 'NumReads', 'N50Length', and 'Longest' from object to numeric
data['NumReads'] = pd.to_numeric(data['NumReads'], errors='coerce')
data['N50Length'] = pd.to_numeric(data['N50Length'], errors='coerce')
data['Longest'] = pd.to_numeric(data['Longest'], errors='coerce')

data.dtypes

Sample_ID            object
num_unique_taxa       int64
Location           category
Sampler              object
Repeat                int64
Flow_rate             int64
Sample_length         int64
Air_volume            int64
NumReads              int64
MeanLength          float64
N50Length             int64
Shortest              int64
Longest               int64
DNA_yield           float64
Insect               object
dtype: object

In [71]:
# Define features and target variable
X = data.drop(['num_unique_taxa'], axis=1)
y = data['num_unique_taxa']

In [77]:
# Define preprocessing for numerical and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), ['Repeat', 'Flow_rate', 'Sample_length', 'Air_volume', 'NumReads', 'MeanLength', 'N50Length', 'Shortest', 'Longest', 'DNA_yield']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Location', 'Sampler', 'Insect', 'Sample_ID'])
    ],
    remainder='passthrough'
)

In [60]:
# Create the pipeline with preprocessor and Random Forest model
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('classifier', RandomForestRegressor(n_estimators=100, random_state=42))
# ])

# # Split data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train the model
# pipeline.fit(X_train, y_train)

# # Predict on the test set
# y_pred = pipeline.predict(X_test)

# # Evaluate the model
# mse = mean_squared_error(y_test, y_pred)
# print(f"Mean Squared Error: {mse}")

#high MSE indicates that the model is not very accurate, probably because I don't have much data

Mean Squared Error: 632.1850374999999


In [82]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), ['Flow_rate', 'Sample_length', 'Air_volume', 'NumReads', 'MeanLength', 'N50Length', 'DNA_yield']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Location', 'Sampler'])
    ],
    remainder='passthrough'
)

# Specify columns to include (ignore 'Repeat & Sample_ID', 'Insect', Longest & Shortest)
columns_to_include = ['Flow_rate', 'Sample_length', 'Air_volume', 'NumReads', 'MeanLength', 'N50Length', 'DNA_yield', 'Location', 'Sampler', ]

# Filter columns
X_filtered = X[columns_to_include]

Running the model to create a visual tree - slightly different way to before

In [83]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.tree import export_graphviz
from numpy import mean, std
import pydot
from subprocess import check_call

# Define features and target variable
X = data.drop(['num_unique_taxa'], axis=1)
y = data['num_unique_taxa']

# Preprocess the data - using the preprocessing from the previous code
X_preprocessed = preprocessor.fit_transform(X_filtered) #Taking a subset of columns specified above
#X_preprocessed = preprocessor.fit_transform(X)#Will take all columns in

# Define the model
model = RandomForestRegressor()

# Fit the model
model.fit(X_preprocessed, y)

# Evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X_preprocessed, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')

# Report performance
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

# Export one of the trees in the Random Forest
estimator = model.estimators_[0]

# Get feature names from preprocessing
feature_names = (preprocessor.transformers_[0][2] +
                  list(preprocessor.transformers_[1][1].get_feature_names_out()))

# Export as dot file
export_graphviz(estimator, out_file='../Images/tree.dot',
                feature_names=feature_names,
                rounded=True, proportion=False,
                precision=2, filled=True)

# Convert dot file to PNG image
check_call(['dot', '-Tpng', '../Images/tree.dot', '-o', '../Images/random_forest_tree.png'])


MAE: -19.673 (9.460)


0